# DataFrameIt - Exemplo 07: Usando Polars DataFrame

Este notebook demonstra como usar o DataFrameIt com Polars, uma alternativa moderna e performática ao Pandas.

**Conceitos demonstrados:**
- Conversão automática Polars ↔ Pandas
- Todas as funcionalidades funcionam com Polars
- Comparação de performance Pandas vs Polars
- Quando usar Polars

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bdcdo/dataframeit/blob/main/example/06_polars.ipynb)

## 1. Instalação

In [ ]:
!pip install -q dataframeit[google,polars]

## 2. Configuração da API Key

In [ ]:
import os

try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    print("API Key carregada dos Secrets do Colab")
except:
    pass

# os.environ['GOOGLE_API_KEY'] = 'sua-chave-aqui'

if 'GOOGLE_API_KEY' not in os.environ:
    print("AVISO: Configure sua GOOGLE_API_KEY antes de continuar")
else:
    print("API Key configurada com sucesso!")

## 3. Imports

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
import polars as pl
import pandas as pd
from dataframeit import dataframeit
import time

## 4. Definir Modelo Pydantic

In [ ]:
class TaskCategory(BaseModel):
    """Classificação de tarefas."""

    categoria: Literal['urgente', 'importante', 'rotina', 'delegavel'] = Field(
        ...,
        description="Categoria da tarefa baseada na matriz de Eisenhower"
    )

    estimativa_tempo: Literal['rapido', 'medio', 'demorado'] = Field(
        ...,
        description="Estimativa de tempo necessário"
    )

## 5. Definir Template

In [ ]:
TEMPLATE = """
Classifique a tarefa abaixo:

{texto}

Use a matriz de Eisenhower para classificação.
"""

## 6. Criar Dados com Polars

In [ ]:
# Criar dados diretamente com Polars
dados = {
    'task_id': [1, 2, 3, 4, 5],
    'texto': [
        "Responder email urgente do cliente sobre bug crítico",
        "Revisar documentação do projeto",
        "Planejar sprint da próxima semana",
        "Organizar arquivos pessoais",
        "Resolver problema de segurança reportado"
    ]
}

# CRIAR POLARS DATAFRAME
df_polars = pl.DataFrame(dados)

print(f"DataFrame Polars criado:")
print(f"  Tipo: {type(df_polars)}")
print(f"  Linhas: {len(df_polars)}")
print(f"  Colunas: {df_polars.columns}")

df_polars

## 7. Processar com Polars

In [ ]:
print("Processando com Polars...")

start_time = time.time()

# O DataFrameIt aceita Polars diretamente!
# Conversão interna é automática
resultado_polars = dataframeit(
    df_polars,                    # Polars DataFrame
    TaskCategory,
    TEMPLATE,
    text_column='texto'
)

elapsed_time = time.time() - start_time

print(f"\nProcessamento concluído em {elapsed_time:.2f}s")
print(f"Tipo do resultado: {type(resultado_polars)}")
print("O DataFrame retornado está no formato ORIGINAL (Polars)")

## 8. Visualizar Resultados Polars

In [ ]:
# Selecionar colunas relevantes (sintaxe Polars)
colunas = ['task_id', 'categoria', 'estimativa_tempo']
resultado_polars.select(colunas)

In [ ]:
# Estatísticas com Polars
print("Distribuição por categoria:")
print(resultado_polars.group_by('categoria').count())

print("\nDistribuição por tempo estimado:")
print(resultado_polars.group_by('estimativa_tempo').count())

## 9. Comparação: Pandas vs Polars

In [ ]:
# Criar mesmo dataset com Pandas
df_pandas = pd.DataFrame(dados)

print("Processando com Pandas...")
start_pandas = time.time()

resultado_pandas = dataframeit(
    df_pandas,
    TaskCategory,
    TEMPLATE,
    text_column='texto'
)

elapsed_pandas = time.time() - start_pandas

print(f"\nTempo Pandas: {elapsed_pandas:.2f}s")
print(f"Tempo Polars: {elapsed_time:.2f}s")

**Nota:** Para datasets pequenos, a diferença de performance é negligível pois o gargalo é a chamada à API do LLM, não o processamento do DataFrame.

A vantagem do Polars aparece em:
- Pré-processamento de dados grandes
- Operações complexas de transformação
- Leitura de arquivos muito grandes
- Joins e agregações em datasets gigantes

## 10. Recursos Avançados do Polars

### Lazy Evaluation
```python
# Criar LazyFrame (não executado ainda)
lf = pl.scan_csv('dados.csv')

# Aplicar transformações (ainda não executado)
lf_filtered = lf.filter(pl.col('status').is_null())

# Processar com DataFrameIt
resultado = dataframeit(lf_filtered.collect(), Model, TEMPLATE)
```

### Expressões Poderosas
```python
# Filtrar antes de processar
df_filtrado = df_polars.filter(
    pl.col('texto').str.len_chars() > 10
)

resultado = dataframeit(df_filtrado, Model, TEMPLATE)
```

### Performance em Larga Escala
```python
# Para datasets gigantes (>1M linhas)
df_large = pl.read_parquet('dados_grandes.parquet')  # Muito rápido!
df_filtered = df_large.filter(pl.col('precisa_processar') == True)
resultado = dataframeit(df_filtered, Model, TEMPLATE, resume=True)
```

## 11. Quando Usar Polars?

### Use Polars quando:
- Trabalha com datasets grandes (>100k linhas)
- Precisa de performance máxima
- Usa operações complexas de transformação
- Quer sintaxe mais moderna e expressiva
- Trabalha com arquivos Parquet
- Precisa de lazy evaluation

### Use Pandas quando:
- Trabalha com datasets pequenos/médios (<100k linhas)
- Já tem código legado em Pandas
- Precisa de compatibilidade com bibliotecas que só suportam Pandas
- Quer documentação e exemplos mais abundantes

## 12. Salvando Resultados

Polars suporta múltiplos formatos de forma eficiente.

In [ ]:
# Descomente para salvar

# CSV
# resultado_polars.write_csv('resultado_polars.csv')

# Parquet (formato recomendado para grandes volumes)
# resultado_polars.write_parquet('resultado_polars.parquet')

# JSON
# resultado_polars.write_json('resultado_polars.json')

# Para Excel, converta para Pandas temporariamente
# resultado_polars.to_pandas().to_excel('resultado_polars.xlsx', index=False)

print("Parquet é o formato recomendado para datasets grandes:")
print("  - Compressão eficiente")
print("  - Leitura/escrita muito rápida")
print("  - Preserva tipos de dados")
print("  - Suporte para colunas aninhadas")

## Resumo

- DataFrameIt suporta Polars nativamente
- Conversão Polars ↔ Pandas é automática e transparente
- Resultado retorna no formato original (Polars in, Polars out)
- Todas as funcionalidades funcionam igualmente (resume, retry, etc.)
- Para datasets grandes, Polars oferece melhor performance
- A escolha entre Pandas e Polars não afeta o uso do DataFrameIt

---

## Próximos Passos

- [07_multiple_data_types.ipynb](07_multiple_data_types.ipynb) - Diferentes tipos de dados
- [08_rate_limiting.ipynb](08_rate_limiting.ipynb) - Configurar rate limiting